In [1]:
import json
import os
import sys

import json
from tqdm import tqdm

def load_jsonl_to_memory(filepath, fraction=4):
    # Determine the total number of lines to calculate the size of the fraction
    with open(filepath, 'r', encoding='utf-8') as file:
        total_lines = sum(1 for _ in file)
    
    # Calculate the number of lines to process based on the fraction
    lines_to_process = total_lines // fraction
    
    # Preallocate the list with None values for the fraction of data
    data = [None] * lines_to_process
    
    with open(filepath, 'r', encoding='utf-8') as file:
        processed_lines = 0  # Keep track of how many lines have been processed
        for index, line in enumerate(tqdm(file, total=total_lines, desc="Processing")):
            if index % fraction == 0:  # Process only every fraction-th line
                # Parse the JSON content from the line and add it to the data list
                data[processed_lines] = json.loads(line)
                processed_lines += 1
                if processed_lines >= lines_to_process:
                    break  # Stop if we've processed the intended number of lines
    
    return data

data = load_jsonl_to_memory('/mnt/datassd/processed_file.jsonl')

Processing: 100%|█████████▉| 53209640/53209647 [00:33<00:00, 1581152.42it/s]


In [2]:
# data in GB
sys.getsizeof(data) / 1024**3

0.09911073744297028

In [3]:
import numpy as np
len(data)

13302411

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming 'data' is your list of dictionaries
titles = [item['title'] for item in data]
view_counts = np.array([item['view_count'] for item in data])

# Parameters for tokenization and padding
vocab_size = 10000  # Adjust based on your dataset
max_length = 100  # Adjust based on the length of your titles
padding_type = 'post'
trunc_type = 'post'

# Initialize and fit the tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(titles)

# Convert titles to sequences and pad them
sequences = tokenizer.texts_to_sequences(titles)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, view_counts, test_size=0.2, random_state=42)


2024-03-05 12:55:09.427256: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 12:55:09.427328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 12:55:09.448544: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 12:55:09.503422: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-05 12:55:10.795227: W tensorflow/compiler/tf2

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam

vocab_size = 10000  # Adjust based on your vocabulary size
embedding_dim = 256
max_length = 100  # Adjust based on your titles' maximum length

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=4),
    LSTM(128, return_sequences=True),
    GlobalMaxPooling1D(),
    Dense(256, activation='relu'),
    Dense(1, activation='linear')  # Linear activation for a regression task
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_squared_error')

model.summary()


2024-03-05 12:59:24.794973: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-05 12:59:24.894243: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-05 12:59:24.894475: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 256)          2560000   
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           163968    
                                                                 
 max_pooling1d (MaxPooling1  (None, 24, 128)           0         
 D)                                                              
                                                                 
 lstm (LSTM)                 (None, 24, 128)           131584    
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 256)               3

In [6]:
del data


In [7]:
# Assuming X_train, y_train are your training data and labels, respectively
model.fit(X_train, y_train, validation_split=0.1, epochs=10, batch_size=32)


2024-03-05 13:00:18.542208: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3831094000 exceeds 10% of free system memory.


Epoch 1/10


2024-03-05 13:00:22.997683: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-03-05 13:00:23.150873: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-05 13:00:23.941312: I external/local_xla/xla/service/service.cc:168] XLA service 0x70f5c114f9d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-05 13:00:23.941337: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2024-03-05 13:00:23.963607: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1709665224.078136   83444 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


299305/299305 [==============================] - ETA: 0s - loss: 11772652158976.0000

2024-03-05 13:24:42.017489: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 425677200 exceeds 10% of free system memory.


299305/299305 [==============================] - 1521s 5ms/step - loss: 11772652158976.0000 - val_loss: 4562952388608.0000
Epoch 2/10
293394/299305 [============================>.] - ETA: 28s - loss: 11806864048128.0000

In [ ]:
# Predict view counts for the test set
predictions = model.predict(X_test, verbose=1)

# Optionally, compare the first few predictions to the actual view counts
for i in range(10):  # Display first 10 predictions
    print(f"Predicted view count: {predictions[i]}, Actual view count: {y_test[i]}")


In [ ]:
# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test, verbose=1)

# If you have specified any metrics when compiling the model, they will also be returned
# Example: model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# Then you can unpack the results as follows:
# loss, mae = model.evaluate(X_test, y_test, verbose=1)

print(f"Test Loss: {loss}")
# If applicable: print(f"Test MAE: {mae}")
